<a href="https://colab.research.google.com/github/jack0928/computer-vision-fundamentals/blob/main/04_mnist_cnn_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow.keras.datasets as ds

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense
from tensorflow.keras.optimizers import Adam


# =========================
# 1. MNIST 데이터 로드
# =========================
# x_train, x_test: (샘플 수, 28, 28) 형태의 흑백 이미지
# y_train, y_test: 0~9 정수 레이블
(x_train, y_train), (x_test, y_test) = ds.mnist.load_data()


# =========================
# 2. 데이터 전처리
# =========================
# CNN 입력을 위해 (높이, 너비, 채널) 구조로 변환
# 흑백 이미지이므로 채널 = 1
x_train = x_train.reshape(60000, 28, 28, 1)
x_test = x_test.reshape(10000, 28, 28, 1)

# 픽셀 값을 0~1 범위로 정규화
x_train = x_train.astype(np.float32) / 255.0
x_test = x_test.astype(np.float32) / 255.0

# 정수 레이블을 one-hot 벡터로 변환
# 예: 3 → [0,0,0,1,0,0,0,0,0,0]
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)


# =========================
# 3. CNN 모델 정의
# =========================
cnn = Sequential()

# --- 첫 번째 Conv 블록 ---
# 32개의 3x3 필터로 특징 추출
cnn.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
cnn.add(Conv2D(32, (3, 3), activation='relu'))

# 공간 크기를 절반으로 줄여 연산량 감소
cnn.add(MaxPooling2D(pool_size=(2, 2)))

# 과적합 방지를 위해 일부 뉴런 무작위 제거
cnn.add(Dropout(0.25))


# --- 두 번째 Conv 블록 ---
# 더 많은 필터(64개)로 고수준 특징 추출
cnn.add(Conv2D(64, (3, 3), activation='relu'))
cnn.add(Conv2D(64, (3, 3), activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2)))
cnn.add(Dropout(0.25))


# =========================
# 4. 분류기(Dense Layer)
# =========================
# 2D feature map → 1D 벡터로 변환
cnn.add(Flatten())

# 완전연결층: 고차원 특징 조합
cnn.add(Dense(units=512, activation='relu'))

# Dense 층에서도 과적합 방지
cnn.add(Dropout(0.5))

# 출력층: 10개 클래스에 대한 확률 출력
cnn.add(Dense(units=10, activation='softmax'))


# =========================
# 5. 모델 컴파일
# =========================
cnn.compile(
    loss='categorical_crossentropy',   # 다중 분류용 손실 함수
    optimizer=Adam(learning_rate=0.001),
    metrics=['accuracy']
)


# =========================
# 6. 모델 학습
# =========================
# batch_size: 한 번에 학습할 샘플 수
# epochs: 전체 데이터를 반복 학습하는 횟수
hist = cnn.fit(
    x_train,
    y_train,
    batch_size=128,
    epochs=100,
    validation_data=(x_test, y_test),
    verbose=2
)


# =========================
# 7. 모델 저장
# =========================
# 학습된 가중치와 구조를 파일로 저장
cnn.save('cnn_v2.h5')


# =========================
# 8. 모델 평가
# =========================
# 테스트 데이터에 대한 최종 성능 측정
res = cnn.evaluate(x_test, y_test, verbose=0)
print('정확률=', res[1] * 100)